In [ ]:
#Install the required libraries!
%pip install -q -U trl numpy torch peft transformers  datasets bitsandbytes wandb

In [ ]:
#Importing required librbries
import logging
import pandas as pd
from scipy.special import softmax
import numpy as np
import torch
from datasets import load_dataset
from transformers import set_seed, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, LlamaTokenizerFast
from peft import LoraConfig, get_peft_model

In [2]:
#This will create a csv file
from get_prompt import pipelined_process
processed_data = pipelined_process('MELD_test_efr.json')
processed_test_data_path = 'test_zds_t3.csv'
processed_data.to_csv(processed_test_data_path, index=False)
processed_data.head()

,text
0,<|system|>In your role as an expert in sentime...
1,<|system|>In your role as an expert in sentime...
2,<|system|>In your role as an expert in sentime...
3,<|system|>In your role as an expert in sentime...
4,<|system|>In your role as an expert in sentime...


In [ ]:
#Keep the config same as below, so we dont encounter any configuration mismatch errors
token='YOUR_HF_API_TOKEN'
Model_Z = "zuhashaik/MELD-EFR-Z"

#Mention the bnb quantization configuraion for the tuned_model
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_qunat_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
)

#Tokenizer for the Model_Z
tokenizer = AutoTokenizer.from_pretrained(Model_Z, token=token)
tokenizer.pad_token = tokenizer.bos_token

#Downloading Model_Z
model = AutoModelForSequenceClassification.from_pretrained(
    Model_Z,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=token
    )
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
#Load the dataset
dataset = load_dataset('csv', data_files={'test': [processed_test_data_path]})

#Let inputs for model_z be tokenized
def tokenize(examples):
        return tokenizer(examples["text"], truncation=True)

test_dataset = dataset['test'].map(tokenize, batched=True)

In [ ]:
import time
from tqdm import tqdm
predicted = []

for text in tqdm(test_dataset['text']):
    # Tokenize the text and create a batch with a single data point
    tokenized = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Perform inference on the single data point
    output = model(**tokenized)
    logits = output.logits
    logits = logits.float()

    # Calculate class probabilities
    class_probabilities = torch.nn.functional.softmax(logits, dim=1)

    predicted.append(class_probabilities)
concatenated_tensor = torch.cat(predicted)
predicted = concatenated_tensor.detach().cpu().numpy()
predicted.shape

In [ ]:
probabilities = softmax(predicted, axis=1)  # Softmax to convert logits to probabilities    
thresholded_predictions = np.argmax(probabilities, axis=1)
df = pd.DataFrame(
    {'predicted_labels':thresholded_predictions}
)
df.head()